In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Personal Practice AI/현우가 시킨 것

/content/drive/MyDrive/Personal Practice AI/현우가 시킨 것


In [48]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

point_img1 = cv2.imread('query1.png', cv2.IMREAD_GRAYSCALE)
point_img2 = cv2.imread('query2.png', cv2.IMREAD_GRAYSCALE)
'''
point_img3 = cv2.imread('Video_APP/img/point_3.png', cv2.IMREAD_GRAYSCALE)
point_img4 = cv2.imread('Video_APP/img/point_4.png', cv2.IMREAD_GRAYSCALE) '''

video = cv2.VideoCapture('sample_vid1.mp4', cv2.IMREAD_GRAYSCALE)

if not video.isOpened():
    print("Could not Open :")
    exit(0)

detector = cv2.SIFT_create()

In [49]:
resize_frame_size = 512
query_img_width = 128

In [50]:
h, w = point_img1.shape
point_img1 = cv2.resize(point_img1, (query_img_width, query_img_width * h // w))
max_height = max(512, query_img_width * h // w)
h, w = point_img2.shape
point_img2 = cv2.resize(point_img2, (query_img_width, query_img_width * h // w))
max_height = max(max_height, query_img_width * h // w)

In [51]:
video_size = (max_height, resize_frame_size+query_img_width)

In [52]:
fps = video.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_out = './output.mp4'

out = cv2.VideoWriter(f'{video_out}', fourcc, fps, (video_size[1], video_size[0]))

In [53]:
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
count_frames = 0

maching_frames = []

while True:
    print('frame :', count_frames, '/', total_frames)
    maching_frames.append(count_frames)
    count_frames += 1

    ret, frame = video.read()

    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (resize_frame_size, resize_frame_size))

    # SIFT 특징점 검출 및 매칭
    kp, des = detector.detectAndCompute(frame, None)
    kp1, des1 = detector.detectAndCompute(point_img1, None)
    kp2, des2 = detector.detectAndCompute(point_img2, None)

    # 특징점 매칭
    bf = cv2.BFMatcher()
    matches1 = bf.knnMatch(des, des1, k=2)
    matches2 = bf.knnMatch(des, des2, k=2)

    # 거리가 가까운 매칭 결과 선택
    good_matches = []
    ratio = 0.5
    good_matches.append(list(first for first,second in matches1 \
                    if first.distance < second.distance * ratio))

    good_matches.append(list(first for first,second in matches2 \
                    if first.distance < second.distance * ratio))

    if len(good_matches[0]) <= 5 and len(good_matches[1]) <= 5:
        print('Cannot find Homography')
        frame = np.pad(frame, [(0, video_size[0]-resize_frame_size), (0, video_size[1]-resize_frame_size)], mode='constant')
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
        out.write(np.array(frame).reshape(frame.shape[0], frame.shape[1] , 3))
        continue

    else:
        print('Find Homography')

        print('-- good matches --')
        print('total matches :', len(matches1))

        # 4 -> point_num
        for i in range(2):
            print(i,'- matches :',len(good_matches[i]))
        for i in range(2):
            if len(good_matches[i]) > 5:
                if i == 0:
                    kp_point = kp1
                    matches_point = matches1
                    good_matches_point = good_matches[0]
                    point = point_img1
                elif i == 1 and len(good_matches[1]) > len(good_matches[0]):
                    kp_point = kp2
                    matches_point = matches2
                    good_matches_point = good_matches[1]
                    point = point_img2

                src_pts = np.float32([ kp[m.queryIdx].pt for m in good_matches[i] ])
                dst_pts = np.float32([ kp_point[m.trainIdx].pt for m in good_matches[i] ])

                break

        mtrx, mask = cv2.findHomography(src_pts, dst_pts)

        h,w, = frame.shape[:2]

        pts = np.float32([ [[0,0]],[[0,h-1]],[[w-1,h-1]],[[w-1,0]] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts, mtrx)

        point_img1 = cv2.polylines(point_img1,[np.int32(dst)],True,255,3, cv2.LINE_AA)

        res = cv2.drawMatches(frame, kp, point, kp_point, good_matches_point, None, \
                            flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

        h, w = res.shape[:2]
        if (video_size[0]-h) * (video_size[1]-w) != 0:
          res = np.pad(res, [(0, video_size[0]-h), (0, video_size[1]-w)], mode='constant')
        out.write(np.array(res).reshape(res.shape[0], res.shape[1], 3))

        # plt.figure(figsize=(15, 10))
        # plt.imshow(res)
        # plt.axis('off')
        # plt.show()

out.release()

frame : 0 / 1102
Cannot find Homography
frame : 1 / 1102
Cannot find Homography
frame : 2 / 1102
Cannot find Homography
frame : 3 / 1102
Cannot find Homography
frame : 4 / 1102
Cannot find Homography
frame : 5 / 1102
Cannot find Homography
frame : 6 / 1102
Cannot find Homography
frame : 7 / 1102
Cannot find Homography
frame : 8 / 1102
Cannot find Homography
frame : 9 / 1102
Cannot find Homography
frame : 10 / 1102
Cannot find Homography
frame : 11 / 1102
Cannot find Homography
frame : 12 / 1102
Cannot find Homography
frame : 13 / 1102
Cannot find Homography
frame : 14 / 1102
Cannot find Homography
frame : 15 / 1102
Cannot find Homography
frame : 16 / 1102
Cannot find Homography
frame : 17 / 1102
Cannot find Homography
frame : 18 / 1102
Cannot find Homography
frame : 19 / 1102
Cannot find Homography
frame : 20 / 1102
Cannot find Homography
frame : 21 / 1102
Cannot find Homography
frame : 22 / 1102
Cannot find Homography
frame : 23 / 1102
Cannot find Homography
frame : 24 / 1102
Cannot f